In [4]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import joblib

# 1. Muat dataset pelatihan dari file 'your_file_updated.csv'
df_train = pd.read_csv('your_file_updated.csv')
print("Dataset pelatihan berhasil dimuat.")

# 2. Pra-pemrosesan Data
# Tentukan fitur (X) dan target (y)
features = ['calories', 'proteins', 'fat', 'carbohydrate']
target = 'Food Score'

# Isi nilai yang kosong (NaN) dengan 0
df_train = df_train.fillna(0)

# Ubah target (Food Score) dari string ke angka (0, 1, 2, 3, ...)
label_encoder = preprocessing.LabelEncoder()
df_train[target] = label_encoder.fit_transform(df_train[target])

# Pisahkan fitur dan target
X = df_train[features]
y = df_train[target]

# 3. Menskalakan Data dan Menyimpan Scaler
PredictorScaler = StandardScaler()
X_scaled = PredictorScaler.fit_transform(X)

# Simpan scaler yang sudah terlatih
joblib.dump(PredictorScaler, 'scaler.joblib')
print("Scaler berhasil disimpan.")

# 4. Melatih Model SVM dan Menyimpannya
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
svm_model_linear = SVC(kernel='linear', C=1).fit(X_train, y_train)

# Simpan model yang sudah terlatih
joblib.dump(svm_model_linear, 'svm_model_linear.joblib')
print("Model SVM berhasil disimpan.")

# 5. Evaluasi Model (Opsional)
svm_predictions = svm_model_linear.predict(X_test)
accuracy = accuracy_score(y_test, svm_predictions)
print(f"\nAccuracy: {accuracy:.4f}")

Dataset pelatihan berhasil dimuat.
Scaler berhasil disimpan.
Model SVM berhasil disimpan.

Accuracy: 0.7946
